# Simulating an Interchange with Amber

<details>
    <summary><small>▼ Click here for dependency installation instructions</small></summary>
    The simplest way to install dependencies is to use the Interchange examples environment. From the root of the cloned openff-interchange repository:
    
    conda env create --name interchange-examples --file devtools/conda-envs/examples_env.yaml 
    conda activate interchange-examples
    pip install -e .
    cd examples/openmm
    jupyter notebook protein_ligand.ipynb
    
</details>

In this example, we'll quickly construct an `Interchange` and then run a simulation in Amber. 

We need an `Interchange` to get started, so let's put that together quickly. For more explanation on this process, take a look at the [packed_box] and [protein_ligand] examples.

[packed_box]: https://github.com/openforcefield/openff-interchange/tree/main/examples/packed_box
[protein_ligand]: https://github.com/openforcefield/openff-interchange/tree/main/examples/protein_ligand

In [5]:
import time

from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.utils import get_data_file_path
from openff.units import unit
from openmm.app import PDBFile
import openmm
import nglview
import mdtraj as md
from pandas import read_csv

from openff.interchange import Interchange

# Read a structure from the Toolkit's test suite into a Topology
pdbfile = PDBFile(
    get_data_file_path("systems/packmol_boxes/propane_methane_butanol_0.2_0.3_0.5.pdb")
)
molecules = [Molecule.from_smiles(smi) for smi in ["CCC", "C", "CCCCO"]]
off_topology = Topology.from_openmm(pdbfile.topology, unique_molecules=molecules)

# Construct the Interchange with the OpenFF "Sage" force field
interchange = Interchange.from_smirnoff(
    force_field=ForceField("openff-2.0.0.offxml"), 
    topology=off_topology,
)
interchange.positions = pdbfile.positions

Tada! A beautiful solvent system:

In [6]:
interchange.visualize("nglview")

NGLWidget()


## Run a simulation

We need Amber input files to run our simulation. The `.prmtop` stores the topology and paramaters, while the `.inpcrd` file is for coordinates:

In [7]:
interchange.to_prmtop("amber.prmtop")
interchange.to_inpcrd("amber.inpcrd")

We also need an input file to describe the simulation parameters:

In [44]:
amber_in = """Basic Amber control file
&cntrl
  imin=0,                ! Run molecular dynamics.
  ntx=1,                 ! Take positions from input and generate velocities
  nstlim=1000,           ! Number of MD-steps to be performed.
  dt=0.002,              ! Time step (ps)
  tempi=300.0,           ! Initial temperature for velocity generation
  temp0=300.0,           ! Thermostat temperature
  cut=10.0,              ! vdW cutoff (Å)
  igb=0,                 ! Don't use a Generalized Born model
  ntt=3, gamma_ln=20.0,   ! Temperature scaling using Langevin dynamics with the collision frequency in gamma_ln (ps−1)
  ntp=0,                 ! No pressure scaling
  iwrap=1,               ! Wrap trajectory coordinates to stay in box
  ioutfm=1,              ! Write out netcdf trajectory
  ntwx=1,                ! Frequency to write coordinates
  ntpr=50                ! Frequency to log energy info
/
"""
with open("amber.in", "w") as f:
    f.write(amber_in)

Run the simulation with Sander:

In [45]:
!sander                \
    -O                 \
    -i amber.in        \
    -p amber.prmtop    \
    -c amber.inpcrd    \
    -x trajectory.nc

And finally we can visualize!

In [46]:
traj = md.load("trajectory.nc", top=md.load_prmtop("amber.prmtop"))
nglview.show_mdtraj(traj)

NGLWidget(max_frame=999)